<a href="https://colab.research.google.com/github/miiiingi/dacon_sentiment/blob/main/dacon_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')
my_folder = '/gdrive/MyDrive/ColabNotebooks/dacon_senti/dataset/dataset'

Mounted at /gdrive/


In [ ]:
import pandas as pd
import numpy as np
!pip install konlpy
from konlpy.tag import Okt
from tqdm import tqdm
trainset = pd.read_csv(f'{my_folder}/train.csv')
valset = trainset.iloc[int(len(trainset) * 0.5):, :]
trainset = trainset.iloc[: int(len(trainset) * 0.5), :]
testset = pd.read_csv(f'{my_folder}/test.csv')

     |████████████████████████████████| 19.4 MB 40.3 MB/s 
     |████████████████████████████████| 448 kB 42.7 MB/s 


훈련 데이터셋 전처리

In [ ]:
trainset['document'] = trainset['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
trainset['document'] = trainset['document'].str.replace('^ +', "")
trainset['document'].replace('', np.nan, inplace=True)
trainset = trainset.dropna(how = 'any')

        id                               document  label
0        1              영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐      0
1        2              히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯      1
2        3       괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요      1
3        4              아무래도 년도지난작품이라 지금보기는너무유치하다      0
4        5  지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다      0
...    ...                                    ...    ...
2495  2496             조디 포스터와 깁슨은 역시 대단하다 감탄 그자체      1
2496  2497  편은편을위해 존재했다고보여질정도로 편은 상당한재미와 완성도를보여준다      1
2497  2498               인생영화중 하나그리고 가슴을 파고드는 엔딩송      1
2498  2499              뭐지 이건 뭘 이야기하고 싶은건지 평점너무높다      0
2499  2500                시간만 때우자 했을때도 주저해야 할 드라마      0

[2495 rows x 3 columns]


검증 데이터셋 전처리

In [ ]:
valset['document'] = valset['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
valset['document'] = valset['document'].str.replace('^ +', "")
valset['document'].replace('', np.nan, inplace=True)
valset = valset.dropna(how = 'any')

        id                                document  label
2500  2501        쌍칼형님 연기 너무 못해서 정말 실망였어요 연기력 키우세요      0
2501  2502     작품성은 있을지 몰라도 ㅡㅡ 이런 애들장난 보고싶은게 아니었는데      0
2502  2503                아무리 애들영화라지만 스토리가 너무 성의없다      0
2503  2504                        영화로 만드는 자체가 무리였어      0
2504  2505                이 영화를 보고 새로운 삶을 살게 되었습니다      1
...    ...                                     ...    ...
4995  4996                 좋은 배우들로 류영화를 찍은 안타까운 영화      0
4996  4997  진짜 드럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼게 아닌영화      0
4997  4998                   가장 실망스러운 영화 지금까지 본영화중      0
4998  4999           이런 평점 테러 네이버에서 좀 막아야 하는 것 아닌가      1
4999  5000       여주인공이 전작에서는 귀여웠는데 여기선 완전 망했네 실망이다      0

[2495 rows x 3 columns]


테스트 데이터 전처리 > 특수 문자를 없애고 공백을 NaN으로 처리하니까 NaN값이 많이 생겼음.... >> 이것을 어떻게 처리 할 것 인가에 대해 생각해보자

In [ ]:
testset['document'] = testset['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
testset['document'] = testset['document'].str.replace('^ +', "")
testset['document'].replace('', np.nan, inplace=True)
print(testset.loc[testset['document'].isnull() == True, :])
testset = testset.dropna(how = 'any')

        id document
2        3      NaN
458    459      NaN
662    663      NaN
756    757      NaN
1023  1024      NaN
1313  1314      NaN
1325  1326      NaN
1386  1387      NaN
1392  1393      NaN
2294  2295      NaN
2599  2600      NaN
3060  3061      NaN
3473  3474      NaN
3654  3655      NaN
3857  3858      NaN
3997  3998      NaN
4001  4002      NaN
4707  4708      NaN


훈련 데이터셋에 대해 불용어 처리하고 토큰화 진행

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '이라', '까지의', '까지', '에서', '것', '게']
okt = Okt()
trainset_sentence = list()
for sentence in tqdm(trainset['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    trainset_sentence.append(stopwords_removed_sentence)

100%|██████████| 2495/2495 [00:17<00:00, 143.93it/s]


In [ ]:
trainset_processed = pd.DataFrame({'document' : trainset_sentence, 'label' : trainset['label']})
print(trainset_processed)

                                               document  label
0              [영상, 이나, 음악, 이쁘다, 해도, 미화, 시키다, 불륜, 일, 뿐]      0
1                             [히치콕, 영화, 보다, 분명, 박수, 치다]      1
2               [괜찮다, 음악, 영화, 또, 나오다, 따뜻하다, 겨울, 되다, 같다]      1
3                  [아무래도, 년도, 지난, 작품, 지금, 보기, 너무, 유치하다]      0
4               [지금, 영화, 그렇다, 영화, 역시, 일본, 대한, 미화, 여전하다]      0
...                                                 ...    ...
2495                 [조디, 포스터, 깁슨, 역시, 대단하다, 감탄, 그, 자체]      1
2496  [편, 편, 을, 위해, 존재, 보이다, 정도, 로, 편, 상당하다, 재미, 완성,...      1
2497           [인생, 영화, 중, 하나, 그리고, 가슴, 을, 파고들다, 엔딩, 송]      1
2498        [뭐, 지, 이건, 뭘, 이야기, 하고, 싶다, 건지다, 평점, 너무, 높다]      0
2499                           [시간, 만, 때우다, 때, 주저, 드라마]      0

[2495 rows x 2 columns]


검증 데이터셋에 대해 불용어 처리하고 토큰화 진행

In [ ]:
valset_sentence = list()
okt = Okt()
for sentence in tqdm(valset['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    valset_sentence.append(stopwords_removed_sentence)

100%|██████████| 2495/2495 [00:06<00:00, 392.47it/s]


In [ ]:
valset_processed = pd.DataFrame({'document' : valset_sentence, 'label' : valset['label']})
print(valset_processed)

                                               document  label
2500          [쌍칼, 형님, 연기, 너무, 못, 정말, 실망, 이다, 연기력, 키우다]      0
2501     [작품, 성은, 있다, 모르다, ㅡㅡ, 이렇다, 애, 장난, 보고, 싶다, 아니다]      0
2502               [아무리, 애, 영화, 라지, 만, 스토리, 너무, 성의, 없다]      0
2503                                [영화로, 만들다, 자체, 무리다]      0
2504                        [영화, 보고, 새롭다, 삶, 을, 살, 되어다]      1
...                                                 ...    ...
4995                   [좋다, 배우, 로, 류, 영화, 찍다, 안타깝다, 영화]      0
4996  [진짜, 드럽다, 재미없다, 에드워드, 호퍼, 그림, 배경, 빼다, 볼, 아니다, 영화]      0
4997                    [가장, 실망, 스러운, 영화, 지금, 본, 영화, 중]      0
4998                        [이렇다, 평점, 테러, 네이버, 막다, 아니다]      1
4999     [여, 주인공, 전작, 에서는, 귀엽다, 여, 기선, 완전, 망하다, 실망, 이다]      0

[2495 rows x 2 columns]


In [ ]:
testset_sentence = list()
for sentence in tqdm(testset['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    testset_sentence.append(stopwords_removed_sentence)

100%|██████████| 4982/4982 [00:11<00:00, 422.49it/s]


In [ ]:
testset_processed = pd.DataFrame({'document' : testset_sentence})
print(testset_processed)

                                            document
0                             [시간, 때우다, 좋다, 영화, 지루함]
1       [훈훈하다, 정이, 느껴지다, 영화, 가족, 끼리, 드라마, 보다, 보다, 딱]
2                           [멋있다, 영화, 이다, 잊다, 수, 없다]
3                               [너무, 감동, 적다, 펑펑, 울다]
4     [어이, 김구라, 자세, 똑바로, 앉다, 방송, 마다, 왜케, 삐, 딱하다, 앉다]
...                                              ...
4977    [연, 계, 제대로, 안되다, 뭔가, 부족하다, 느낌, 들다, 찝찝하다, 영화]
4978                [내생, 최악, 영화, 중, 하나, 졸리다, 미치다, 뻔]
4979            [지금, 까지, 나오다, 드라마, 중, 최고, 이다, 점, 만점]
4980              [영화, 값, 엄청나다, 공연, 을, 보다, 게, 웬, 횡재]
4981                      [올해, 엔, 롯데, 자이언츠, 꼭, 우승하다]

[4982 rows x 1 columns]


등장 단어 데이터 셋 만들기

In [ ]:
import collections
wordset = collections.defaultdict(int)
wordset_freq = collections.defaultdict(int)
for sentence in tqdm(trainset_processed['document']) :
  for word in sentence : 
    wordset[word] += 1
for k, v in wordset.items() : 
  if wordset[k] > 5 :
    wordset_freq[k] = v
wordset['<unk>'] = 0
wordset['<pad>'] = 1 

100%|██████████| 4990/4990 [00:00<00:00, 241307.64it/s]


단어 임베딩 벡터 만들기

In [ ]:
from torch import nn as nn
embed_layer = nn.Embedding(num_embeddings=len(wordset_freq),
                           embedding_dim = 16,
                           padding_idx = 1) 

Parameter containing:
tensor([[ 0.5725, -0.0116, -0.8020,  ..., -1.0997, -1.2119,  0.8811],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-1.7805,  0.0376, -1.4147,  ...,  0.8625,  1.0358,  1.2306],
        ...,
        [ 0.1353,  1.4251, -0.2256,  ...,  0.2817,  0.7274, -0.0612],
        [ 0.1899,  0.0749, -0.2070,  ..., -2.5309, -0.6005,  0.6978],
        [-0.2626,  1.8274,  0.1970,  ..., -0.9226, -1.8234, -1.7732]],
       requires_grad=True)


데이터셋 분리하기

In [ ]:
from torchtext import data
print(dir(data))
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'bleu_score', 'custom_replace', 'datasets_utils', 'filter_wikipedia_xml', 'functional', 'generate_sp_model', 'get_tokenizer', 'interleave_keys', 'load_sp_model', 'metrics', 'numericalize_tokens_from_iterator', 'sentencepiece_numericalizer', 'sentencepiece_tokenizer', 'simple_space_split', 'to_map_style_dataset', 'utils']


AttributeError: ignored